In [15]:
import networkx as nx
import networkx.algorithms.approximation as approx
from networkx.algorithms.tree.mst import random_spanning_tree
import tsplib95
import numpy as np
import pulp
from math import exp, log as ln, ceil
import math
import time


In [16]:
def solve(k):
    problem = tsplib95.load("data/p43.atsp")
    G = problem.get_graph()
    G.is_directed() 


    # only keep weights: (i,j,{'weight': weight, 'is_fixed": True}) -> G[i][j] = weight
    for u, v, data in G.edges(data=True):
        weight = data.get('weight', 0)  
        G[u][v].clear()  
        G[u][v]['weight'] = weight

    # making sure G is complete

    for i in G.nodes:
        for j in G.nodes:
            if i != j and not G.has_edge(i, j):
                G.add_edge(i, j, weight=9999)
                print(f"[!] added edge {i} {j} with weight 9999")
                
    G = G.subgraph(list(G.nodes)[:k])
    # print edges in a matrix


    start = time.time()


    def find_violated_cut(G, x_vals):
        """
        Finds a violated cut constraint using a max-flow/min-cut approach.

        Parameters:
            G (nx.DiGraph): The directed graph.
            x_vals (dict): Current LP solution values for edges.

        Returns:
            U (set): A subset of nodes violating the cut constraint, or None if no violation.
        """
        residual = nx.DiGraph()
        for (u, v), val in x_vals.items():
            residual.add_edge(u, v, capacity=val)

        # check for violated cuts for each pair of source and sink nodes
        for source in G.nodes:
            for sink in G.nodes:
                if source != sink:
                    cut_value, (reachable, non_reachable) = nx.minimum_cut(
                        residual, source, sink, capacity='capacity'
                    )
                    
                    if cut_value < 1:
                        return reachable  # violated subset as reachable nodes
        return None  

    def HK(G):
        """
        Compute the Held-Karp relaxation for the ATSP using a separation oracle.

        Parameters:
            G (nx.DiGraph): Directed graph with 'weight' attributes on edges.

        Returns:
            objective (float): The optimal objective value.
            z_star (dict): Symmetrized solution of the Held-Karp relaxation.
        """
        if not G.is_directed():
            raise ValueError("Graph must be a directed graph (DiGraph).")

        V = list(G.nodes)
        A = list(G.edges)
        c = nx.get_edge_attributes(G, 'weight')

        prob = pulp.LpProblem("Held-Karp_Relaxation", pulp.LpMinimize)
        x = pulp.LpVariable.dicts("x", A, lowBound=0, cat='Continuous')

        prob += pulp.lpSum([c[arc] * x[arc] for arc in A]), "Total_Cost"

        # flow conservation constraints (3.3)
        for v in V:
            prob += pulp.lpSum([x[(v, w)] for w in G.successors(v)]) == 1, f"Outflow_{v}"
            prob += pulp.lpSum([x[(u, v)] for u in G.predecessors(v)]) == 1, f"Inflow_{v}"

        constraint_counter = 0
        while True:
            prob.solve()

            # find support graph
            x_vals = {arc: (x[arc].varValue or 0) for arc in A}

            # use separation oracle to find violated cut if any
            violated_cut = find_violated_cut(G, x_vals)
            if violated_cut is None:
                break  # good

            # hash names
            delta_plus_U = [(u, v) for u in violated_cut for v in G.successors(u) if v not in violated_cut]
            prob += pulp.lpSum([x[arc] for arc in delta_plus_U]) >= 1, f"Cut_{constraint_counter}"
            constraint_counter += 1  


        x_star = {arc: (x[arc].varValue or 0) for arc in A}

        # symmetrize solution to get z_star (3.5)
        z_star = {}
        scale_factor = (len(V) - 1) / len(V)  # (n-1)/n scaling factor from paper
        for (u, v) in A:
            frequency = x_star[(u, v)] + x_star.get((v, u), 0)
            if frequency > 0:
                z_star[(u, v)] = scale_factor * frequency

        # sanity check for connectivity
        support_graph = nx.Graph()
        for (u, v), val in z_star.items():
            if val > 0:
                support_graph.add_edge(u, v)

        if not nx.is_connected(support_graph):
            raise ValueError("Held-Karp solution produced disconnected support graph!")

        return pulp.value(prob.objective), z_star


    HK_opt, z_star = HK(G)



    def q_(G, gamma):
        node_list = list(G.nodes())
        node_index = {node: i for i, node in enumerate(node_list)}
        
        for u, v, d in G.edges(data=True):
            d["lambda"] = exp(gamma[(u, v)])
        
        L = nx.laplacian_matrix(G, weight='lambda').toarray()

        L_pinv = np.linalg.pinv(L)
        
        q = {}
        for u, v, d in G.edges(data=True):
            ui, vi = node_index[u], node_index[v]
            R_uv = L_pinv[ui, ui] + L_pinv[vi, vi] - 2 * L_pinv[ui, vi]
            q[(u, v)] = d['lambda'] * R_uv
        
        return q



    def spanning_tree_distribution(G, z):
        gamma = {arc: 0 for arc in G.edges()}

        EPSILON = 0.2

        while True:
            # search for an edge with q_e > (1 + epsilon) * z_e
            for e in gamma.keys():
                q_e = q_(G, gamma)[e]
                z_e = z[e]
                if q_e > (1 + EPSILON) * z_e:
                    # lemma 7.1
                    delta = ln(
                        (q_e * (1 - (1 + EPSILON / 2) * z_e))
                        / ((1 - q_e) * (1 + EPSILON / 2) * z_e)
                    )
                    # check that delta had the desired effect on q
                    # we decrease gamma_e in order to decrease qe(gamma) to (1 +epsilon/2) * ze
                    gamma[e] -= delta
                    new_q_e = q_(G, gamma)[e]
                    desired_q_e = (1 + EPSILON / 2) * z_e
                    if round(new_q_e, 8) != round(desired_q_e, 8):
                        raise nx.NetworkXError(
                            f"Unable to modify probability for edge ({u}, {v})"
                        )
            q = q_(G, gamma)
            if all(q[e] <= (1 + EPSILON) * z[e] for e in G.edges()):
                break

        # remove lambda from edge attribute: not needed anymore
        for _, _, d in G.edges(data=True):
            if "lambda" in d:
                del d["lambda"]

        return gamma


    z_support = nx.MultiGraph()
    for u, v in z_star:
        if (u, v) not in z_support.edges:
            edge_weight = min(G[u][v]['weight'], G[v][u]['weight'])
            z_support.add_edge(u, v, weight=edge_weight)



    gamma = spanning_tree_distribution(z_support, z_star)


    z_support = nx.Graph(z_support)
    lambda_dict = {(u, v): exp(gamma[(u, v)]) for u, v in z_support.edges()}
    nx.set_edge_attributes(z_support, lambda_dict, "weight")
    # del gamma, lambda_dict

    # sample 2 * ceil( ln(n) ) spanning trees and record the minimum one
    minimum_sampled_tree = None
    minimum_sampled_tree_weight = math.inf
    for _ in range(2 * ceil(ln(G.number_of_nodes()))):
        sampled_tree = random_spanning_tree(z_support, "weight", seed=None)
        sampled_tree_weight = sampled_tree.size(weight)
        if sampled_tree_weight < minimum_sampled_tree_weight:
            minimum_sampled_tree = sampled_tree.copy()
            minimum_sampled_tree_weight = sampled_tree_weight

    # orient the edge
    t_star = nx.MultiDiGraph()
    for u, v, d in minimum_sampled_tree.edges(data=weight):
        if d == G[u][v]['weight']:
            t_star.add_edge(u, v, weight=d)
        else:
            t_star.add_edge(v, u, weight=d)

    node_demands = {n: t_star.out_degree(n) - t_star.in_degree(n) for n in t_star}
    nx.set_node_attributes(G, node_demands, "demand")

    # demand graph based on in/out degree imbalance
    flow_dict = nx.min_cost_flow(G, "demand")

    for source, values in flow_dict.items():
        for target in values:
            if (source, target) not in t_star.edges and values[target] > 0:
                # IF values[target] > 0 we have to add that many edges
                for _ in range(values[target]):
                    t_star.add_edge(source, target)
    circuit = nx.eulerian_circuit(t_star, source=source)





    def shortcutting(circuit):
        """Remove duplicate nodes in the path"""
        nodes = []
        for u, v in circuit:
            if v in nodes:
                continue
            if not nodes:
                nodes.append(u)
            nodes.append(v)
        nodes.append(nodes[0])
        return nodes


        
    ans = shortcutting(circuit)


    total_cost = 0
    for i in range(len(ans)-1):
        total_cost += G[ans[i]][ans[i+1]]['weight']

    print(f"total cost: {total_cost}")
    print(ans)
    total_time = time.time() - start

    return total_cost, total_time


In [17]:
import json

try:
    with open('ans.json', 'r') as file:
        ans = json.load(file)
except FileNotFoundError:
    ans = {}

for i in range(34, 0, -1):
    for repeat in range(5):
        total_cost, runtime = solve(int(i))
        key = str(i) 
        if key not in ans:
            ans[key] = {"total_cost": [], "runtime": []}
        if total_cost not in ans[key]["total_cost"]:
            ans[key]["total_cost"].append(total_cost)
        if runtime not in ans[key]["runtime"]:
            ans[key]["runtime"].append(runtime)
        with open('ans.json', 'w') as file:
            json.dump(ans, file, indent=4)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/FakeYQL/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4s/2ly7t48j4r57l9zmt1db4qgw0000gn/T/00829673d77e4272827531337e4da5f6-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/4s/2ly7t48j4r57l9zmt1db4qgw0000gn/T/00829673d77e4272827531337e4da5f6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 85 COLUMNS
At line 4792 RHS
At line 4873 BOUNDS
At line 4874 ENDATA
Problem MODEL has 80 rows, 1600 columns and 3200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 80 (0) rows, 1600 (0) columns and 3200 (0) elements
Perturbing problem by 0.001% of 5104 - largest nonzero change 0.0030295582 ( 0.25462867%) - largest zero change 0.0025527287
0  Obj 0 Primal inf 79.999992 (80)
58  Obj 0.071328331
Optimal - objective value 0
Optimal objective 0 - 58 iterati